In [1]:
import numpy as np
import tensorflow as tf

/home/psaint/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/psaint/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/psaint/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/psaint/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: 

In [2]:
class FirstColumn(tf.keras.Model): 
    def __init__(self,hidden_dims,activation_fns,name = None):
        super(FirstColumn,self).__init__(name = name)
        
        assert isinstance(hidden_dims,list)
        assert isinstance(activation_fns,list)
        assert len(hidden_dims) == len(activation_fns)
        
        self.columns = [[tf.keras.layers.Dense(dim, activation=fn,
                                        name='fully_connected_' + str(i),trainable = True) for i,(dim,fn) in enumerate(zip(hidden_dims,activation_fns))]]
        self.width = 1
        self.depth = len(hidden_dims)
    def call(self,x):
        for layer in self.columns[0]: 
            x = layer(x)  
        return x  
    
class ProgNet(tf.keras.Model):
    def __init__(self,prev_net,hidden_dims,activation_fns,name=None):
        super(ProgNet,self).__init__(name = name)
        assert isinstance(hidden_dims,list)
        assert isinstance(activation_fns,list)
        assert len(hidden_dims) == len(activation_fns)
        assert isinstance(prev_net,(ProgNet,FirstColumn))
        
        self.depth = len(hidden_dims)
        self.columns = []
        self.U = []
        self.activation_fns = activation_fns
        
      
        assert (len(prev_net.columns[-1]) == len(hidden_dims))
        self.columns = prev_net.columns
        self.columns.append([tf.keras.layers.Dense(dim,
                                    name="fully_connected_" + str(i),trainable = True) for i,dim in enumerate(hidden_dims)])
        self.width = prev_net.width + 1
        self.U = [[tf.keras.layers.Dense(dim,use_bias = False,name='U_'+str(i)+'_' + str(j),trainable = True) for i,dim in enumerate(hidden_dims)] for j in range(self.width-1)]
            
             
    def call(self,x):
        x = [x]*self.width
        for i in range(self.depth-1):
            x = [self.columns[j][i](x[j]) for j in range(self.width)]
            print(self.U)
            Uh = [self.U[j][i](x[j]) for j in range(self.width - 1)]
            x[-1] = self.activation_fns[i](tf.add_n(Uh) + x[-1])
        return self.columns[-1][-1](x[-1])
          
        

In [3]:
#data
size = 100
bound = 10
x_train = np.random.uniform(0,bound,size).reshape(size,1)
y_train = np.floor(x_train).astype('int').reshape(size)

In [4]:
hidden_dims = [64,128,64,10]
act_fns = [tf.keras.activations.relu for i in range(4)]

net1 = FirstColumn(hidden_dims,act_fns,name = 'Col1')
net2 = ProgNet(net1,hidden_dims,act_fns,name='Col2')
net3 = ProgNet(net2,hidden_dims,act_fns,name='Col3')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [5]:
x = tf.placeholder(dtype = tf.float32, shape = [None,1])
y = tf.placeholder(dtype = tf.int32, shape = [None])

In [6]:
loss1 = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, 
                                                                    logits = net1(x)))
loss2 = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, 
                                                                    logits = net2(x)))
loss3 = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, 
                                                                    logits = net3(x)))

train_vars1 = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,
                                     "Col1")
train_vars2 = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,
                                     "Col2")

train_vars3 = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,
                                      "Col3")
# Define an optimizer 
train_op1 = tf.train.AdamOptimizer(learning_rate=0.01).minimize(loss1,var_list = train_vars1)
train_op2 = tf.train.AdamOptimizer(learning_rate=0.01).minimize(loss2,var_list = train_vars2)
train_op3 = tf.train.AdamOptimizer(learning_rate=0.01).minimize(loss3,var_list = train_vars3)

ListWrapper([ListWrapper([<tensorflow.python.keras.layers.core.Dense object at 0x7f6892361da0>, <tensorflow.python.keras.layers.core.Dense object at 0x7f6891aeb0b8>, <tensorflow.python.keras.layers.core.Dense object at 0x7f6891aeb390>, <tensorflow.python.keras.layers.core.Dense object at 0x7f6891aeb630>])])
ListWrapper([ListWrapper([<tensorflow.python.keras.layers.core.Dense object at 0x7f6892361da0>, <tensorflow.python.keras.layers.core.Dense object at 0x7f6891aeb0b8>, <tensorflow.python.keras.layers.core.Dense object at 0x7f6891aeb390>, <tensorflow.python.keras.layers.core.Dense object at 0x7f6891aeb630>])])
ListWrapper([ListWrapper([<tensorflow.python.keras.layers.core.Dense object at 0x7f6892361da0>, <tensorflow.python.keras.layers.core.Dense object at 0x7f6891aeb0b8>, <tensorflow.python.keras.layers.core.Dense object at 0x7f6891aeb390>, <tensorflow.python.keras.layers.core.Dense object at 0x7f6891aeb630>])])
ListWrapper([ListWrapper([<tensorflow.python.keras.layers.core.Dense obje

In [7]:
with tf.Session() as sess: 
    writer = tf.summary.FileWriter('logs', sess.graph)
    sess.run(tf.global_variables_initializer())
    sess.run(train_op1,feed_dict={x:x_train,y:y_train})
    sess.run(train_op2,feed_dict={x:x_train,y:y_train})
    sess.run(train_op3,feed_dict={x:x_train,y:y_train})
    writer.close()